In [24]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import sys
sys.path.append('..')
from functions.Multilinear_regression import *
from functions.evaluation import *

Compararemos la implementación de un modelo utilizando un framework y nuestra implementación propia a ver si dan los mismos resultados. 

El dataset se puede encontrar en `Data\salary.csv`. Estos son datos de profesores en una universidad con los cuales vamos a intentar predecir su salario. Como hay algunas variables categoricas vamos a hacer un one-hot encoding con algunas columnas. 

In [4]:
df = pd.read_csv('..\Data\salary.csv')
df = pd.get_dummies(df)
df.head()

,yrs.since.phd,yrs.service,salary,rank_AssocProf,rank_AsstProf,rank_Prof,discipline_A,discipline_B,sex_Female,sex_Male
0,19,18,139750,False,False,True,False,True,False,True
1,20,16,173200,False,False,True,False,True,False,True
2,4,3,79750,False,True,False,False,True,False,True
3,45,39,115000,False,False,True,False,True,False,True
4,40,41,141500,False,False,True,False,True,False,True


Vamos a dividir nuestro dataset en un conjunto de pruebas otro de entrenamiento.

In [6]:

X_train, X_test, y_train, y_test = train_test_split(df.drop(columns = 'salary'), df['salary'], test_size=0.2, random_state=42)

## Modelo estadistico con sklearn

Vamos a crear el modelo 3 veces para comparar todo. Primero utilizando minimos cuadrados para obtener los parámetros óptimos directamente lo cual podemos hacer con la libreria de Sklearn. 

In [12]:
model_stat = LinearRegression()
model_stat.fit(X_train, y_train)
[model_stat.intercept_] +  list(model_stat.coef_)

[93336.8669810207,
 506.78274727016327,
 -307.88522912181566,
 -7064.238273050716,
 -18571.068852445955,
 25635.307125496674,
 -8027.748718966921,
 8027.748718966921,
 -2182.9911443485003,
 2182.9911443485007]

Ahora vamos a graficar las metricas de los modelos para ver como funciona

In [17]:
y_pred_stat = model_stat.predict(X_test)
mse, mae, mape = get_metrics_regression(y_test, y_pred_stat)
print(f'MSE:{mse}   MAE:{mae}   MAPE:{mape}')

MSE:584765286.8530645   MAE:16703.47743313775   MAPE:0.15873779450133146


## Modelo con optimizador en sklearn

Ahora utilizaremos un modelo con optimizador de gradiente descendente estocastico. 

In [21]:
model = SGDRegressor(max_iter=10000)
model.fit(X_train, y_train)
list(model.intercept_) +  list(model.coef_)

[5171472220.348316,
 -6103570733.493881,
 7424601455.622701,
 2848987051.7539425,
 -1692866968.082073,
 4013128334.717951,
 582845373.787163,
 4586403044.60265,
 7545982954.472075,
 -2376734536.082261]

In [20]:
y_pred = model.predict(X_test)
mse, mae, mape = get_metrics_regression(y_test, y_pred_stat)
print(f'MSE:{mse}   MAE:{mae}   MAPE:{mape}')

MSE:584765286.8530645   MAE:16703.47743313775   MAPE:0.15873779450133146


## Modelo Propio

Ahora toca entrenar el modelo

In [29]:
t = multilinear_regression(X_train.values,
                           y_train.values,
                           np.array([10000, 50, 100, 50, 50, 50, 50, 50, 50, 50]),
                           0.0015,
                           10000)
t

array([43576.80690829,   764.33122037,  -375.61001786,  3307.01957558,
       -2694.99465346, 33114.78198617,  8463.35327934, 25213.45362895,
       12466.50048012, 21210.30642817])

Generamos la predicción

In [30]:
y_pred = predict_regresion(t, X_test.values)

Ahora vamos a calcular algunas metricas para medir el rendimiento del modelo

In [31]:
mse, mae, mape = get_metrics_regression(y_test, y_pred)
print(f'MSE:{mse}   MAE:{mae}   MAPE:{mape}')

MSE:601408159.419883   MAE:16924.52640249945   MAPE:0.16123369475333188


## Comparacion

Podemos ver que el mejor modelo resulto ser el estadistico. Sin embargo este tipo de modelos puede llegar a ser lentos dependiendo de los datos que queramos ajustar por lo que no siempre es lo mejor. El que definitivamente fue peor fue el propio :( ya que se tardó más que los otros dos y tuvo un error ligeramente superior. 